In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import random

# Get the CNR-EXT dataset

In [0]:
! wget http://cnrpark.it/dataset/CNR-EXT-Patches-150x150.zip

--2019-09-21 16:03:22--  http://cnrpark.it/dataset/CNR-EXT-Patches-150x150.zip
Resolving cnrpark.it (cnrpark.it)... 146.48.85.22
Connecting to cnrpark.it (cnrpark.it)|146.48.85.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 449502403 (429M) [application/zip]
Saving to: ‘CNR-EXT-Patches-150x150.zip’

CNR-EXT-Patches-150 100%[===================>] 428.68M  91.8MB/s    in 5.7s    

2019-09-21 16:03:28 (75.5 MB/s) - ‘CNR-EXT-Patches-150x150.zip’ saved [449502403/449502403]



In [0]:
 !unzip CNR-EXT-Patches-150x150.zip

#Prepare the dataset

In [0]:
labels = pd.read_csv('LABELS/all.txt', delim_whitespace=True)

In [0]:

for i in range(len(labels)):
  if (i % 15000) == 0:
    print(i)
  oldPath = 'PATCHES/'+ labels.iloc[i, 0];
  newPath1 = 'data/train/full/%d.jpg' % (i,)
  newPath2 = 'data/train/free/%d.jpg' % (i,)
  newPath3 = 'data/val/full/%d.jpg' % (i,)
  newPath4 = 'data/val/free/%d.jpg' % (i,)
  if random.randint(0, 1):
    continue
  if(labels.iloc[i, 1]):
    if random.randint(0, 1) or random.randint(0, 1):
      shutil.move(oldPath, newPath1)
    else:
      shutil.move(oldPath, newPath3)
  else :
    if random.randint(0, 1) or random.randint(0, 1):
      shutil.move(oldPath, newPath2)
    else:
      shutil.move(oldPath, newPath4)
  

0
15000
30000
45000
60000
75000
90000
105000
120000
135000


In [0]:
# Import Libraries
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader, random_split

In [0]:
# Specify transforms using torchvision.transforms as transforms
# library
transformations = transforms.Compose([
    transforms.Resize(75),
    transforms.CenterCrop(75),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [0]:
# Load in each dataset and apply transformations using
# the torchvision.datasets as datasets library
train_data = datasets.ImageFolder("data/train", transform = transformations)
test_data = datasets.ImageFolder("data/val", transform = transformations)

In [0]:
train_loader = torch.utils.data.DataLoader(
                        train_data,
                        batch_size=30,
                        shuffle=True,
                        num_workers=4,
                        pin_memory=True)

val_loader = torch.utils.data.DataLoader(
                        test_data,
                        batch_size=10,
                        shuffle=False,
                        num_workers=4,
                        pin_memory=True)


# GPU

In [0]:
# Find the device available to use using torch library
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Move model to the device specified above
model.to(device)

print(torch.cuda.is_available())

NameError: ignored

# squeeze

In [0]:
# number of classes in the dataset
num_of_output_classes = 2

# HYPER-PARAMETERS

train_only_last_layer = False # boolean variable

In [0]:
# some variables storing dataset info
dset_sizes = {} # empty dict
dset_sizes['train'] = len(train_data)
dset_sizes['val'] = len(test_data)
dset_classes = train_data.classes # number of classes in datasets

In [0]:
# Train on GPU if CUDA is available
use_gpu = torch.cuda.is_available()


"""Use this if you want to visualise the dataset before training"""

# Get a batch of training data
inputs, classes = next(iter(train_loader))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[dset_classes[x] for x in classes])


In [0]:
# UTILITY FUNCTIONS HERE.

import numpy


def exp_lr_scheduler(optimizer, epoch, init_lr=0.001, lr_decay_epoch=7):
    """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
    lr = init_lr * (0.1**(epoch // lr_decay_epoch))

    if epoch % lr_decay_epoch == 0:
        print('LR is set to {}'.format(lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer

# utility function to visualise few images from the dataset
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    plt.imshow(inp)
    # if title is not None:
        # plt.title(title)
    plt.pause(10)  # pause a bit so that plots are updated



def visualize_model(model, num_images=6):
    images_so_far = 0
    fig = plt.figure()

    for i, data in enumerate(val_loader):
        inputs, labels = data
        if use_gpu:
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labels)

        outputs = model(inputs)
        print(outputs)
        _, preds = torch.max(outputs.data, 1)

        for j in range(inputs.size()[0]):
            images_so_far += 1
            ax = plt.subplot(num_images//2, 2, images_so_far)
            ax.axis('off')
            ax.set_title('predicted: {}'.format(dset_classes[labels.data[j]]))
            imshow(inputs.cpu().data[j])

            if images_so_far == num_images:
                return



def train_model(model, criterion, optimizer, lr_scheduler, num_epochs=25):
    since = time.time()

    best_model = model
    best_acc = 0.0
    
    
    counter = 0
    old_loss = 0
    epoch = 0
    # run for given number of epochs
    while epoch <num_epochs:
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        accuracy = 0
        
        a = numpy.array([[epoch, 1]])

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                optimizer = lr_scheduler(optimizer, epoch)
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            if phase == 'train':
                phase_ = train_loader
            else:
                phase_ = val_loader

            # Run through all data in mini batches
            for data in phase_:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs, labels = Variable(inputs.cuda()), \
                        Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward pass
                outputs = model.forward(inputs)
                _, preds = torch.max(outputs.data, 1)

                # calculating the loss
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    old_loss = running_loss
                    loss.backward()
                    optimizer.step()

                # statistics for printing
                running_loss += loss.item()*inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                if phase == 'val' :
                  # Since our model outputs a LogSoftmax, find the real 
                  # percentages by reversing the log function
                  outputs = torch.exp(outputs)
                  # Get the top class of the output
                  top_p, top_class = outputs.topk(1, dim=1)
                  # See how many of the classes were correct?
                  equals = top_class == labels.view(*top_class.shape)
                  # Calculate the mean (get the accuracy for this batch)
                  # and add it to the running accuracy for this epoch
                  accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                
                if phase == 'train':
                  
                  if(counter % 100 == 0) :
                    print('{} Training Loss:{:.6f}'.format(counter, running_loss - old_loss))
                    a = numpy.append(a, [[counter, (running_loss - old_loss)]], axis = 0)
                  counter += 1
            
            
            epoch_loss = running_loss / dset_sizes[phase]
            
          
        epoch_acc = accuracy/len(val_loader)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
          phase, epoch_loss, epoch_acc))

          # save model if it performed better than
          # any other previous model
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model = copy.deepcopy(model)
            
        saveToDrive(save_checkpoint(epoch, model, optimizer, epoch_loss, 'squeeze_sgd1_v'))
        a = numpy.append(a, [[epoch, epoch_loss]], axis = 0)
        a = numpy.append(a, [[epoch, epoch_acc]], axis = 0)
        saveCSV('squeeze_sgd1_v{}.csv'.format(epoch), a)
        
        epoch += 1
        

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    return best_model

In [0]:
# import everything
from __future__ import print_function, division
import torchvision
from torchvision.datasets import ImageFolder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os
######################################################################
# ConvNet training
# ----------------------------------


# using the model defined in the beginning of the program
# Example: model_conv = torchvision.models.resnet18(pretrained=True)

def DefineSqueeze(num_of_output_classes, train_only_last_layer):
  # example
  import torchvision
  model_conv = torchvision.models.squeezenet1_1(pretrained=True) # define model here
  
  if train_only_last_layer is True:
    # turn off backprop update for all the weights in the model
    for param in model_conv.parameters():
        param.requires_grad = False

  # change the last Conv2D layer in case of squeezenet. there is no fc layer in the end.
  num_ftrs = 512
  model_conv.classifier._modules["1"] = nn.Conv2d(512, num_of_output_classes, kernel_size=(1, 1))
  model_conv.classifier._modules["2"] = nn.LogSoftmax(dim=1)

  # because in forward pass, there is a view function call which depends on the final output class size.
  model_conv.num_classes = num_of_output_classes


  model_conv = model_conv.cuda()

  # defining loss criterion, for these
  # models CrossEntropyLoss works the best
  criterion = nn.CrossEntropyLoss()

  # Observe that only parameters of final layer are being optimized as
  # opoosed to before.
  """ Defining an optimiser function here, can use Adam, RMSprop or simple SGD"""
  optimizer_conv = optim.SGD(model_conv.classifier.parameters(), lr=0.001, momentum=0.9)

  return model_conv, optimizer_conv, criterion

model_conv, optimizer_conv, criterion = DefineSqueeze(2, False)

######################################################################
# Train and evaluate
# ^^^^^^^^^^^^^^^^^^
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)

######################################################################

# If we want to visvalise model. then call this function
# visualize_model(model_conv)

# plt.ioff()
# plt.show()

Downloading: "https://download.pytorch.org/models/squeezenet1_1-f364aa15.pth" to /root/.cache/torch/checkpoints/squeezenet1_1-f364aa15.pth
100%|██████████| 4.74M/4.74M [00:00<00:00, 42.4MB/s]

Epoch 0/24
----------
LR is set to 0.001


0 Training Loss:65.376785
100 Training Loss:0.144819
200 Training Loss:0.124399
300 Training Loss:0.092602
400 Training Loss:0.523408
500 Training Loss:0.327716
600 Training Loss:1.379999
700 Training Loss:0.988931
800 Training Loss:0.001445
900 Training Loss:0.015172
1000 Training Loss:0.000035
1100 Training Loss:0.000815
1200 Training Loss:0.001228
1300 Training Loss:0.005157
1400 Training Loss:0.000472
1500 Training Loss:0.002796
1600 Training Loss:0.004320
1700 Training Loss:0.033428
1800 Training Loss:0.062646
1900 Training Loss:0.000698
2000 Training Loss:0.004780
2100 Training Loss:0.005175
2200 Training Loss:0.014380
2300 Training Loss:0.003256
2400 Training Loss:0.558885
2500 Training Loss:0.003832
2600 Training Loss:0.004766
val Loss: 0.0028 Acc: 0.9991
Loss 0.002769
Epoch 1/24
----------
2700 Training Loss:0.001963
2800 Training Loss:0.011353
2900 Training Loss:0.093744
3000 Training Loss:0.007809
3100 Training Loss:0.009337
3200 Training Loss:2.482052
3300 Training Loss:0.2

#Evaluate

In [0]:
# Using our model to predict the label
def predict(image, model):
    # Pass the image through our model
    output = model.cpu().forward(image)
    
    # Reverse the log function in our output
    output = torch.exp(output)
    print(output)
    
    # Get the top predicted class, and the output percentage for
    # that class
    probs, classes = output.topk(1, dim=1)
    return probs.item(), classes.item()

In [0]:
# Show Image
def show_image(image):
    # Convert image to numpy
    image = image.numpy()
    
#     fig = plt.figure(figsize=(25, 4))
#     plt.imshow(np.transpose(image[0], (1, 2, 0)))
    # Un-normalize the image
    image[0] = image[0] * 0.226 + 0.445
    
    # Print the image
    fig = plt.figure(figsize=(25, 4))
    plt.imshow(np.transpose(image[0], (1, 2, 0)))
    plt.pause(0.0001)

In [0]:
# Process Image
    image = process_image('data/val/free/76403.jpg')
    # Give image to model to predict output
    top_prob, top_class = predict(image, model_conv)
    # Show the image
    
    
    show_image(image)
#     ax = plt.subplot(0, 1, i + 1)
#     plt.tight_layout()
    print('Is ' + ('full' if top_class else 'free') + ' with {:.2f}% certainty'.format(top_prob*100))
#     ax.axis('off')
    # Print the results
#     print("The model is ", top_prob*100, "% certain that the image has a predicted class of ", 'full' if top_class else 'free' )
    plt.show()

In [0]:
for i in range(1, 23):
    # Process Image
    image = process_image("Toys/{}.jpg".format(i))
    # Give image to model to predict output
    top_prob, top_class = predict(image, model_conv)
    # Show the image
    
    
    show_image(image)
    ax = plt.subplot(1, 23, i + 1)
#     plt.tight_layout()
    ax.set_title('Is ' + ('full' if top_class else 'free') + ' with {:.2f}% certainty'.format(top_prob*100))
    ax.axis('off')
    # Print the results
#     print("The model is ", top_prob*100, "% certain that the image has a predicted class of ", 'full' if top_class else 'free' )

    if i == 23:
        plt.show()
        break

# (Extra)File Management

In [0]:
import os
import shutil

In [0]:
#list
os.listdir()

['.config',
 'full.zip',
 'dataset.zip',
 'tempfree',
 'data',
 'adc.json',
 'sample_data']

In [0]:
# define the name of the directory to be deleted
path = "dataset"

try:
    shutil.rmtree(path)
except OSError:
    print ("Deletion of the directory %s failed" % path)
else:
    print ("Successfully deleted the directory %s" % path)

Successfully deleted the directory dataset


In [0]:
# define the name of the directory to be created

def createDir(path):
  try:
      os.makedirs(path)
  except OSError:
      print ("Creation of the directory %s failed" % path)
  else:
      print ("Successfully created the directory %s" % path)
      
      
createDir('data/train/full');
createDir('data/train/free');
createDir('data/val/full');
createDir('data/val/free');
createDir('full');
createDir('free');
createDir('tempfree')
createDir('temp')

Successfully created the directory data/train/full
Successfully created the directory data/train/free
Successfully created the directory data/val/full
Successfully created the directory data/val/free
Creation of the directory full failed
Successfully created the directory free
Successfully created the directory tempfree
Creation of the directory temp failed


In [0]:
createDir('temp')

Successfully created the directory temp


In [0]:
# Delete
os.remove('full')

# Save model to drive

In [0]:
def saveToDrive(PATH):
  !pip install -U -q PyDrive
  from pydrive.auth import GoogleAuth
  from pydrive.drive import GoogleDrive 
  from google.colab import auth 
  from oauth2client.client import GoogleCredentials
  
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()                       
  drive = GoogleDrive(gauth)
  
  model_file = drive.CreateFile({'title' : PATH})                      
  model_file.SetContentFile(PATH)                       
  model_file.Upload()
  
  drive.CreateFile({'id': model_file.get('id')})

In [0]:
saveToDrive('squeezenet1_0.pt')

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

     |████████████████████████████████| 993kB 5.0MB/s 


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()                       
drive = GoogleDrive(gauth)

In [0]:
model_file = drive.CreateFile({'title' : 'parking_model.pkl'})                      
model_file.SetContentFile('parking_model.pkl')                       
model_file.Upload()

In [0]:
drive.CreateFile({'id': model_file.get('id')})

GoogleDriveFile({'id': '1bhU3oQ6pmaLRM5GkeY2mcft-Az8oQ0LL'})

#Saving & Loading Model for Inference

In [0]:
torch.save(model.state_dict(), 'parking_model.pkl')

In [0]:
fid = drive.ListFile({'q':"title='parking_model.pkl'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('parking_model.pkl')

In [0]:
model.load_state_dict(torch.load('parking_model.pkl'))
model.eval()

# Save/Load Entire Model

In [0]:
torch.save(model_conv, 'squeezenet_t_v2.pt')

In [0]:
saveToDrive('squeezenet_t_v2.pt')

In [0]:
# Model class must be defined somewhere
model = torch.load('PATH')
model.eval()

# Saving & Loading a General Checkpoint for Inference and/or Resuming Training

In [0]:
def save_checkpoint(epoch, model, optimizer, loss, modelName):
  PATH = modelName + '{}'.format(epoch) + '.pt'
  PATH2 = modelName + '{}'.format(epoch) + '.pkl'
  torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': loss 
              }, PATH)
  
  convert2(PATH, PATH2)
  return PATH

In [0]:
def load_checkpoint(epoch, PATH):
   
  model, optimizer, criterion = DefineSqueeze(2, False)

  checkpoint = torch.load(PATH)
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  print('Loss {:.6f}'.format(checkpoint['loss']))

  model.train()
  return model, optimizer, criterion

In [0]:
saveToDrive(save_checkpoint(epoch, model, optimizer, train_loss/len(train_loader.dataset), 'parking_densenet2_'))

In [0]:
model_conv, optimizer, criterion = load_checkpoint(1, 'squeezenet_tune_1.pt')

Loss 0.068585


In [0]:
def convert(PATH, DEST):
  getFromDrive(PATH)
  model_conv, optimizer, criterion = load_checkpoint(1, PATH)
  torch.save(model_conv, DEST)
  saveToDrive(DEST)

In [0]:
def convert2(PATH, DEST):
  model_conv, optimizer, criterion = load_checkpoint(1, PATH)
  torch.save(model_conv, DEST)
  saveToDrive(DEST)

In [0]:
convert('squeezenet_tune_4.pt', 'squeezenet_t_v6.pt')

Loss 0.077354


# Import Model from Drive

In [0]:
def getFromDrive(name):
  !pip install -U -q PyDrive
  from pydrive.auth import GoogleAuth
  from pydrive.drive import GoogleDrive
  from google.colab import auth
  from oauth2client.client import GoogleCredentials

  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  fid = drive.ListFile({'q':"title='"+ name +"'"}).GetList()[0]['id']
  f = drive.CreateFile({'id': fid})
  f.GetContentFile(name)

In [0]:
getFromDrive('squeezenet_tune_0.pt')

# Tunning dataset

## Get from drive free, and full classes raw files

In [0]:
import cv2

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fid = drive.ListFile({'q':"title='dataset.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('dataset.zip')

!unzip dataset.zip

fid = drive.ListFile({'q':"title='full.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('full.zip')
!unzip full.zip


## Full dataset

clean numbers

In [0]:
import os

DIR = 'full'
TEMP = 'temp'
i = 1
for name in os.listdir(DIR):
	# if os.path.isfile(os.path.join(DIR, name)):
	# 	continue;
	dst = '{}'.format(i) + '.jpg'
	os.rename(os.path.join(DIR, name), os.path.join(TEMP, dst)) 
	i += 1

for name in os.listdir(DIR):
	os.remove(os.path.join(DIR, name))

for name in os.listdir(TEMP):
	os.rename(os.path.join(TEMP, name), os.path.join(DIR, name))

for name in os.listdir(TEMP):
	os.remove(os.path.join(TEMP, name))


*Blur

In [0]:
# loading library 
import cv2 
import numpy as np
import random
import os

DIR = 'full'
size = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
count = size
for i in range(1, size):
	img = cv2.imread('full/{}'.format(i) + '.jpg') 
	  
	# Specify the kernel size. 
	# The greater the size, the more the motion. 
	kernel_size = random.randrange(8, 12)

	# Create the vertical kernel. 
	kernel_v = np.zeros((kernel_size, kernel_size)) 
	  
	# Create a copy of the same for creating the horizontal kernel. 
	kernel_h = np.copy(kernel_v) 
	  
	# Fill the middle row with ones. 
	kernel_v[:, int((kernel_size - 1)/2)] = np.ones(kernel_size) 
	kernel_h[int((kernel_size - 1)/2), :] = np.ones(kernel_size) 
	  
	# Normalize. 
	kernel_v /= kernel_size 
	kernel_h /= kernel_size 
	  
	# Apply the vertical kernel. 
	vertical_mb = cv2.filter2D(img, -1, kernel_v) 
	  
	# Apply the horizontal kernel. 
	horizonal_mb = cv2.filter2D(img, -1, kernel_h) 
	  
	# Save the outputs.
	count += 1
	cv2.imwrite('full/' + str(count) + '.jpg', vertical_mb)
	count += 1
	cv2.imwrite('full/' + str(count) + '.jpg', horizonal_mb)

*gamma

In [0]:
import cv2
import numpy as np

def adjust_gamma(image, gamma=1.0):

  invGamma = 1.0 / gamma
  table = np.array([((i / 255.0) ** invGamma) * 255
    for i in np.arange(0, 256)]).astype("uint8")

  return cv2.LUT(image, table)

cv2.waitKey(0)
cv2.destroyAllWindows()
 
import random
import os

DIR = 'full'
size = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
i = size + 1

for name in os.listdir(DIR):
    if not os.path.isfile(os.path.join(DIR, name)):
      continue
    original = cv2.imread(os.path.join(DIR, name), 1)
    boolean = True if (random.randrange(0, 1)) else False

    gamma = random.uniform(0.1, 0.4)
    adjusted = adjust_gamma(original, gamma=gamma)
    
    changed_name = '{}'.format(i) + '.jpg'
    cv2.imwrite(os.path.join(DIR, changed_name), adjusted)
    i += 1

*flip

In [0]:
import random
import os
import cv2
import numpy as np


DIR = 'full'
size = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
count = size + 1
for i in range(1, size):
	originalImage = cv2.imread('full/{}'.format(i) + '.jpg')
	flipVertical = cv2.flip(originalImage, 0)
	cv2.imwrite('full/' + str(count) + '.jpg', flipVertical)
	count += 1
	flipHorizontal = cv2.flip(originalImage, 1)
	cv2.imwrite('full/' + str(count) + '.jpg', flipHorizontal)
	count += 1
	flipBoth = cv2.flip(originalImage, -1)
	cv2.imwrite('full/' + str(count) + '.jpg', flipBoth)
	count += 1


## Free parking dataset

Initial crop

In [0]:
from PIL import Image
import cv2
import os

RAW_FREE = 'dataset/initfree'
i = 1
for filename in os.listdir(RAW_FREE):
  if not os.path.isfile(os.path.join(RAW_FREE, filename)):
  	continue

  im = Image.open(os.path.join(RAW_FREE, filename))
  im_crop = im.crop((78, 70, 625, 480))
  im_crop.save('tempfree/{}'.format(i) + '.jpg')
  i += 1

Crop with stride

In [0]:
import cv2
import os
import numpy as np

TEMP_FREE = 'tempfree'
intervalX = 10
intervalY = 20
stride = 75
count = 0

for filename in os.listdir(TEMP_FREE):
  if not os.path.isfile(os.path.join(TEMP_FREE, filename)):
  	continue
    
  img = cv2.imread(os.path.join(TEMP_FREE, filename), 1)

  
  # print img.shape

  for i in range(0, img.shape[0] - stride, intervalX):
    for j in range(0, img.shape[1] - stride, intervalY):
        # print j
        cropped_img = img[i:i + stride, j:j + stride]
        count += 1
        cv2.imwrite('free/' + str(count) + '.jpg', cropped_img)


In [0]:
DIR = 'full'
size = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
print(size)

DIR = 'free'
size = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
print(size)

55153
52224


In [0]:
DIR = 'data/train/full'
size = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
length1 = size
DIR = 'data/train/free'
size = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
length2 = size
DIR = 'data/val/full'
size = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
length3 = size
DIR = 'data/val/free'
size = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
length4 = size

i = length1 + length2 + length3 + length4 + 1

DIR = 'full'
for filename in os.listdir(DIR):
  if not os.path.isfile(os.path.join(DIR, filename)):
  	continue
  
  if (i % 15000) == 0:
    print(i)
  oldPath = os.path.join(DIR, filename);
  newPath1 = 'data/train/full/%d.jpg' % (i,)
  newPath3 = 'data/val/full/%d.jpg' % (i,)
#   if random.randint(0, 1):
#     continue
    
  if random.randint(0, 1) or random.randint(0, 1):
    shutil.move(oldPath, newPath1)
  else:
    shutil.move(oldPath, newPath3)
      
  i += 1
  
DIR = 'free'
for filename in os.listdir(DIR):
  if not os.path.isfile(os.path.join(DIR, filename)):
  	continue
  
  if (i % 15000) == 0:
    print(i)
  oldPath = os.path.join(DIR, filename);
  newPath2 = 'data/train/free/%d.jpg' % (i,)
  newPath4 = 'data/val/free/%d.jpg' % (i,)
#   if random.randint(0, 1):
#     continue
    
  if random.randint(0, 1) or random.randint(0, 1):
    shutil.move(oldPath, newPath2)
  else:
    shutil.move(oldPath, newPath4)
      
  i += 1

15000
30000
45000
60000
75000
90000
105000


In [0]:
DIR = 'data/train/full'
size = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
print(size)

DIR = 'data/train/free'
size = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
print(size)

DIR = 'data/val/full'
size = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
print(size)

DIR = 'data/val/free'
size = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
print(size)

41313
39236
13840
12988


In [0]:
import numpy
a = numpy.array([[0, 1]])
 
a = numpy.append(a, [[1, 5]], axis = 0)
a = numpy.append(a, [[2, 0.5]], axis = 0)
a = numpy.append(a, [[3, 3.5]], axis = 0)
a = numpy.append(a, [[4, 1.5]], axis = 0)
a = numpy.append(a, [[5, 0.05]], axis = 0)
a = numpy.append(a, [[6, 0.1]], axis = 0)
a = numpy.append(a, [[7, 0.2]], axis = 0)
numpy.savetxt("foo.csv", a, delimiter=",", fmt='%f')


saveCSVToDrive("foo.csv")

In [0]:
def saveCSVToDrive(PATH):
  !pip install -U -q PyDrive
  from pydrive.auth import GoogleAuth
  from pydrive.drive import GoogleDrive 
  from google.colab import auth 
  from oauth2client.client import GoogleCredentials
  
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()                       
  drive = GoogleDrive(gauth)
  
  model_file = drive.CreateFile({'title' : PATH})                      
  model_file.SetContentFile(PATH)                       
  model_file.Upload()
  
  drive.CreateFile({'id': model_file.get('id')})

In [0]:
def saveCSV(PATH, numArray):
  numpy.savetxt(PATH, numArray, delimiter=",", fmt='%f')
  saveCSVToDrive(PATH)